In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ast  # For safely parsing the string representation of the list

In [6]:
# --- CONFIGURATION ---
RESULTS_FILE = "results_perturb.csv"
HISTORY_DIR = "tests"
PLOTS_DIR = "plots"

# Create main plots directory
if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)

# --- LOAD DATA ---
# Handle path if file is in current directory or inside 'tests'
if os.path.exists(os.path.join(HISTORY_DIR, RESULTS_FILE)):
    result_path = os.path.join(HISTORY_DIR, RESULTS_FILE)
else:
    result_path = RESULTS_FILE

df = pd.read_csv(result_path)

# Helper column for cleaner filenames/titles
df['Config_ID'] = df.apply(lambda x: f"prob_{int(x['N'])}_{x['density']}_{x['alpha']}_{x['beta']}", axis=1)

print(f"Loaded results: {len(df)} runs found.")

# Set global style
sns.set_style("whitegrid")

Loaded results: 24 runs found.


In [7]:
def plot_single_analysis():
    for index, row in df.iterrows():
        # 1. Setup Directory for this problem
        prob_dir_name = f"prob_{int(row['N'])}_{row['density']}_{row['alpha']}_{row['beta']}"
        prob_path = os.path.join(PLOTS_DIR, prob_dir_name)
        
        if not os.path.exists(prob_path):
            os.makedirs(prob_path)
            
        # -----------------------------------------------------
        # PLOT A: Learning Process (Convergence)
        # -----------------------------------------------------
        # Construct history filename. Note: The ThiefSolver saves as prob_d_a_b.csv
        # If you ran multiple N with same d/a/b, this might pick the latest one.
        hist_filename = f"prob_{row['density']}_{row['alpha']}_{row['beta']}.csv"
        hist_file_path = os.path.join(HISTORY_DIR, hist_filename)
        
        if os.path.exists(hist_file_path):
            hist_df = pd.read_csv(hist_file_path)
            
            plt.figure(figsize=(10, 6))
            
            # Plot Lines
            plt.plot(hist_df['Generation'], hist_df['Avg_Cost'], label='Average Population Cost', color='orange', alpha=0.5, linewidth=1.5)
            plt.plot(hist_df['Generation'], hist_df['Best_Cost'], label='Best Solution Cost', color='#1f77b4', linewidth=2.5)
            plt.axhline(y=row['Baseline'], color='#d62728', linestyle='--', linewidth=2, label='Baseline (Greedy)')
            
            # Highlight Perturbations (Cataclysm)
            # We assume a perturbation happens if Stagnation_Count drops significantly (e.g. from >=40 to 0)
            perturbations = hist_df[hist_df['Stagnation_Count'].diff() < -40]
            for gen in perturbations['Generation']:
                plt.axvline(x=gen, color='purple', linestyle=':', alpha=0.6, label='Perturbation Triggered' if gen == perturbations['Generation'].iloc[0] else "")
                
            plt.title(f"Learning Process: N={int(row['N'])}, d={row['density']}, α={row['alpha']}, β={row['beta']}", fontsize=14)
            plt.xlabel("Generation")
            plt.ylabel("Total Cost")
            plt.legend()
            plt.tight_layout()
            
            # Save Plot A
            plt.savefig(os.path.join(prob_path, "A_convergence.png"), dpi=300)
            plt.close()
        else:
            print(f"Warning: History file not found for {prob_dir_name} (Skipping Plot A)")

        # -----------------------------------------------------
        # PLOT B: Thief's Burden (Weight Profile)
        # -----------------------------------------------------
        # Parse the path string into a list of tuples
        try:
            path_data = ast.literal_eval(row['Path'])
            
            weights = []
            current_w = 0.0
            steps = []
            
            for i, (node, gold) in enumerate(path_data):
                steps.append(i)
                # If we are at base (node 0), weight resets AFTER arrival (effectively 0 for next step)
                # But for the plot, we want to show the drop. 
                if node == 0:
                    current_w = 0.0
                else:
                    current_w += gold
                weights.append(current_w)
                
            plt.figure(figsize=(10, 5))
            
            # Fill area to look like a mountain profile
            plt.fill_between(steps, weights, color='#2ca02c', alpha=0.3)
            plt.plot(steps, weights, color='#2ca02c', linewidth=1.5)
            
            plt.title(f"Thief's Burden (Weight Profile)\nSteps: {len(steps)} | Final Cost: {row['Cost']:,.0f}", fontsize=14)
            plt.xlabel("Step Number (Physical Path)")
            plt.ylabel("Current Weight Carried")
            plt.grid(True, linestyle=':', alpha=0.6)
            
            # Save Plot B
            plt.savefig(os.path.join(prob_path, "B_weight_profile.png"), dpi=300)
            plt.close()
            
        except Exception as e:
            print(f"Error parsing path for {prob_dir_name}: {e}")

    print("Single Problem Analysis plots created.")